In [1]:
!pip install pymysql > /dev/null

In [2]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving mysql.json to mysql.json


In [3]:
import json
with open(filename) as fp:
    config_str = fp.read()
config = json.loads(config_str)

In [4]:
import pymysql
conn = pymysql.connect(**config)

In [5]:
import random
random.seed(2021)

## Products Table

In [6]:
sql = '''
CREATE TABLE if NOT EXISTS products(
    pid INT PRIMARY KEY AUTO_INCREMENT,
    pname VARCHAR(40) NOT NULL,
    pprice INT UNSIGNED NOT NULL,
    pcategory VARCHAR(20) NOT NULL,
    pcost INT UNSIGNED NOT NULL
) AUTO_INCREMENT=11;
'''

In [7]:
cur = conn.cursor()
cur.execute(sql)

0

In [8]:
cpu_goods = ['1050T', 'FX-8300', 'FX-9590', '1700X', '2700X',
             '3300X', '3700X', '3800X', '5600X', '5900X']
gpu_goods = ['RX560', 'RX570', 'RX580', 'RX590', 'RX5600',
             'RX5600XT', 'RX5700XT', 'RX6700XT', 'RX6800', 'RX6900XT']
mb_goods = ['970A', '990FX', 'A320', 'A520', 'B350',
            'B450', 'B550', 'X370', 'X470', 'X570']
ctg_names = ['CPU', 'GPU', 'MB']

In [9]:
sql = '''
INSERT INTO products(pname, pprice, pcategory, pcost)
VALUES(%s, %s, %s, %s);
'''

In [10]:
for i, item_list in enumerate([cpu_goods, gpu_goods, mb_goods]):
    for item in item_list:
        price = random.randint(100, 999) * 100
        rate = random.randint(85, 98)
        cost = int(round(price * rate / 1000) * 10)
        cur.execute(sql, (item, price, ctg_names[i], cost))
conn.commit()

## Sales Table

In [11]:
sql = '''
CREATE TABLE if NOT EXISTS sales(
    sid INT PRIMARY KEY AUTO_INCREMENT,
    sdate DATE NOT NULL,
    scompany VARCHAR(20) NOT NULL,
    spid INT NOT NULL,
    sunit INT NOT NULL,
    FOREIGN KEY(spid) REFERENCES products(pid)
) AUTO_INCREMENT=101;
'''

In [12]:
cur = conn.cursor()
cur.execute(sql)

0

In [13]:
calender = {1:31, 2:29, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
company_list = ['AMD', 'INTEL', 'SAMSUNG', 'TSMC', 'NVIDIA']

In [14]:
for i in range(10):
    date = f'2020-1-{random.randint(1, 31)}'
    company = company_list[random.randint(0, 4)]
    pid = random.randint(11, 40)
    unit = random.randint(1, 20)
    print(date, company, pid, unit)

2020-1-10 INTEL 19 1
2020-1-1 NVIDIA 35 8
2020-1-7 INTEL 37 8
2020-1-13 TSMC 29 12
2020-1-23 SAMSUNG 12 9
2020-1-3 AMD 16 19
2020-1-15 INTEL 29 5
2020-1-21 TSMC 23 9
2020-1-14 INTEL 31 14
2020-1-5 NVIDIA 37 16


In [15]:
sql = '''
INSERT INTO sales VALUES(default, %s, %s, %s, %s);
'''

In [16]:
for month, day in calender.items():
    for i in range(10):
        date = f'2020-{month}-{random.randint(1, day)}'
        company = company_list[random.randint(0, 4)]
        pid = random.randint(11, 40)
        unit = random.randint(1, 20)
        cur.execute(sql, (date, company, pid, unit))
conn.commit()

## Products Table <- Join -> Sales Table (View)

In [17]:
sql = '''
CREATE VIEW sales_book AS
SELECT sid, sdate, scompany, pid, pname, pcategory,
pprice, sunit, pcost, pprice*sunit AS revenue,
(pprice-pcost)*sunit AS profit
FROM sales
JOIN products
ON sales.spid = products.pid
ORDER BY sdate;
'''

In [18]:
cur = conn.cursor()
cur.execute(sql)

0

### sales_book.csv

In [19]:
sql = '''
SELECT * FROM sales_book;
'''
cur.execute(sql)
results = cur.fetchall()

In [20]:
import pandas as pd
df = pd.DataFrame(results)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,108,2020-01-01,SAMSUNG,34,A520,MB,50400,11,48380,554400,22220
1,102,2020-01-01,TSMC,40,X570,MB,57400,18,52810,1033200,82620
2,104,2020-01-03,NVIDIA,13,FX-9590,CPU,65700,14,58470,919800,101220
3,106,2020-01-06,TSMC,16,3300X,CPU,58500,12,54990,702000,42120
4,107,2020-01-07,INTEL,13,FX-9590,CPU,65700,4,58470,262800,28920


In [21]:
del df[0]
del df[3]

In [22]:
df.columns = ['판매일자', '판매처', '상품명', '카테고리',
              '단가', '수량', '원가', '매출액', '이익']
df.head()

,판매일자,판매처,상품명,카테고리,단가,수량,원가,매출액,이익
0,2020-01-01,SAMSUNG,A520,MB,50400,11,48380,554400,22220
1,2020-01-01,TSMC,X570,MB,57400,18,52810,1033200,82620
2,2020-01-03,NVIDIA,FX-9590,CPU,65700,14,58470,919800,101220
3,2020-01-06,TSMC,3300X,CPU,58500,12,54990,702000,42120
4,2020-01-07,INTEL,FX-9590,CPU,65700,4,58470,262800,28920


In [23]:
df.to_csv('sales_book.csv', index=False)

In [24]:
cur.close()
conn.close()